In [3]:
from numba import jit
import random

@jit(nopython=True)
def monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [4]:
import numpy as np
monte_carlo_pi(10000000)

3.141916

In [69]:
from numba import cuda, float32
import time
import math
@cuda.jit
def increment_by_one(an_array):
    # Thread id in a 1D block
    tx = cuda.threadIdx.x
    # Block id in a 1D grid
    ty = cuda.blockIdx.x
    # Block width, i.e. number of threads per block
    bw = cuda.blockDim.x
    # Compute flattened index inside the array
    pos = tx + ty * bw
    for i in range(1000000):
        if pos < an_array.size:  # Check array boundaries
            an_array[pos] = math.sqrt(abs(an_array[pos]))

an_array = np.random.normal(0,1,size=2**15)
print(an_array[:10])
threadsperblock = 64
blockspergrid = (an_array.size + (threadsperblock - 1)) // threadsperblock
t1 = time.time()
increment_by_one[blockspergrid, threadsperblock](an_array)
t2 = time.time()
print(an_array[:10])
print(t2-t1)

[-0.63200664  0.35868749 -0.88308861  0.85521855  0.40723921 -0.41009166
 -0.04985493  0.0773229   1.69922401  0.79534132]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
14.17832064628601


In [35]:
@jit(nopython=True)
def inc(an_array):
    for i in range(1000000):
        an_array = np.sqrt(np.abs(an_array))
    return an_array

t1 = time.time()
an_array = inc(np.random.normal(0,1,size=2**15))
t2 = time.time()
print(t2-t1)

24.22390604019165


In [39]:
an_array = np.random.normal(0,1,size=2**15)
t1 = time.time()
an_array = inc(an_array)
t2 = time.time()
print(t2-t1)

24.132588863372803


In [190]:
import math
import matplotlib.pyplot as plt
from numba import cuda, float64
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float64, xoroshiro128p_normal_float64
# plt.scatter(x[:,0],x[:,1],alpha=0.02)
# np.random.seed(0)
@cuda.jit
def mcmc(data, output, rng_states,n_iter):
    shared = cuda.shared.array(shape=(2**11,), dtype=float64)
    tx = cuda.threadIdx.x  # Thread ID
    ty = cuda.blockIdx.x  # Block ID
    bw = cuda.blockDim.x  # Block Size
    idx = bw*ty+tx
    
    theta = (0.,0.)
    x = data[idx]
    log_p_i = -(((theta[0]-x[0])**2)/(2*0.1) + ((theta[1]-x[1])**2)/(2*0.1))
    shared[tx] = log_p_i
    cuda.syncthreads()
    
    s = bw//2
    while s>0:
        if tx < s:
            shared[tx] += shared[tx+s]
        cuda.syncthreads()
        s>>=1
    
    log_p = shared[0]  # ?
    
    for i in range(n_iter):
        if tx == 0:
#             theta_ = (theta[0] + 0.1*xoroshiro128p_normal_float64(rng_states, ty), theta[1] + 0.1*xoroshiro128p_normal_float64(rng_states, ty))
            shared[0] = theta[0] + 0.05*xoroshiro128p_normal_float64(rng_states, idx)
            shared[1] = theta[1] + 0.05*xoroshiro128p_normal_float64(rng_states, idx)
        cuda.syncthreads()
        theta_ = shared[0],shared[1]
        
            
        
        log_p_i = -(((theta_[0]-x[0])**2)/(2*0.1) + ((theta_[1]-x[1])**2)/(2*0.1))
        shared[tx] = log_p_i
        cuda.syncthreads()

        s = bw//2
        while s>0:
            if tx < s:
                shared[tx] += shared[tx+s]
            cuda.syncthreads()
            s>>=1
            
        log_p_ = shared[0]
        
        alpha = math.exp(min(0,log_p_-log_p))
        if tx == 0:
            shared[0] = xoroshiro128p_uniform_float64(rng_states, idx)
        cuda.syncthreads()
        u = shared[0]
        
        if u < alpha:
            theta = theta_
            log_p = log_p_
        
        # ?
        if tx == 0:
            output[i] = theta
        

n_iter = 100000

# Prior distribution
theta_0 =  np.random.multivariate_normal([1,1],cov=[[1, 0.5],[0.5, 1]])
data = np.random.multivariate_normal(theta_0,cov=[[0.1, 0],[0, 0.1]],size=2**25)
threadsperblock = 1024
blockspergrid = (data.shape[0] + (threadsperblock - 1)) // threadsperblock

rng_states = create_xoroshiro128p_states(threadsperblock * blockspergrid, seed=1)

output = np.empty((n_iter,2))

t1 = time.time()
mcmc[blockspergrid, threadsperblock](data,output,rng_states, n_iter)
t2 = time.time()
print(t2-t1)
# print(output)
# print(theta_0)
output_ = np.empty((n_iter,3))
output_[:,:2] = output
output_[:,2] = np.arange(0,n_iter)

plt.figure()
plt.scatter(data[:,0],data[:,1],alpha=0.2)
plt.scatter(theta_0[0],theta_0[1],color='red')
plt.show()

fig = plt.figure()
plt.scatter(output_[n_iter//2:,0],output_[n_iter//2:,1],c=output_[n_iter//2:,2],alpha=0.1)
plt.scatter(theta_0[0],theta_0[1],color='red')
plt.colorbar()
plt.show()

KeyboardInterrupt: 

In [86]:
from scipy.stats import multivariate_normal as mn
mn.pdf([0,0],mean=[1,1],cov=[[1, 0.5],[0.5, 1]])

0.09435389770895924

In [187]:
fig.savefig('mcmc2.png')